In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
'''
Author: error: error: git config user.name & please set dead value or install git && error: git config user.email & please set dead value or install git & please set dead value or install git
Date: 2025-05-16 10:33:15
LastEditors: error: error: git config user.name & please set dead value or install git && error: git config user.email & please set dead value or install git & please set dead value or install git
LastEditTime: 2025-08-13 17:58:08
FilePath: \Project_Business\风控\2.model_func.ipynb
Description: 这是默认设置,请设置`customMade`, 打开koroFileHeader查看配置 进行设置: https://github.com/OBKoro1/koro1FileHeader/wiki/%E9%85%8D%E7%BD%AE
'''
def data_report(df, numeric_missing_value = [np.nan, -999, -1111], string_missing_value = ['']):
    '''
    df：包含需要做统计分析变量的数据集
    numeric_missing_value：连续型缺失值，list
    string_missing_value：字符型缺失值，list
    '''
    dtypes_dict = df.dtypes.to_dict()
    dtypes = pd.DataFrame()
    dtypes['var'] = dtypes_dict.keys()
    dtypes['types'] = dtypes_dict.values()
    dtypes = dtypes[~dtypes['var'].isin(fixed_var)]
    
    df_string = dtypes[dtypes['types'] == 'object']
    df_numeric = dtypes[dtypes['types'] != 'object']
    
    if df_numeric.shape[0] == 0:
        print ('连续型变量个数为0，不做统计分析')
    else:
        numeric_cols = list(df_numeric['var'])
        numeric_desc = pd.DataFrame()
        for col in numeric_cols:
            df1 = df.loc[df[col].isin(numeric_missing_value), [col]]
            df2 = df.loc[~df[col].isin(numeric_missing_value), [col]]
            
            if df2.shape[0] != 0:
                quantiles_dict = df2[col].quantile([0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99]).to_dict()
                quantiles = pd.DataFrame()
                quantiles['quantiles'] = quantiles_dict.keys()
                quantiles[col] = quantiles_dict.values()
                quantiles = quantiles.set_index('quantiles')
                quantiles_T = quantiles.T
                quantiles_T.rename(columns = {0.01:'1%', 0.05:'5%', 0.25:'25%', 0.5:'50%', 0.75:'75%', 0.95:'95%',0.99:'99%'}, inplace = True)
            
                quantiles_T['skew'] = df2[col].skew() #偏度
                quantiles_T['kurt'] = df2[col].kurt()
                quantiles_T['missing'] = df1.shape[0]
                quantiles_T['count'] = df.shape[0]
                quantiles_T['missing_rate'] = quantiles_T['missing'] *1.0 / quantiles_T['count']
                quantiles_T['unique'] = len(set(df2[col]))
                quantiles_T['min'] = df2[col].min()
                quantiles_T['max'] = df2[col].max()
                quantiles_T['mode'] = df2.loc[df2[col] == df2[col].mode()[0], col].count()
                quantiles_T['mode_rate'] = quantiles_T['mode'] *1.0 / df2.shape[0]
                quantiles_T = quantiles_T[['count', 'missing', 'missing_rate', 'unique', 'mode', 'mode_rate', 'min', '1%', '5%', '25%',
                                           '50%', '75%', '95%', '99%', 'max', 'skew', 'kurt']]                 
                numeric_desc = pd.concat([numeric_desc, quantiles_T])
            else:
                print ("{} 无正常取值".format(col))
        numeric_desc = numeric_desc.reset_index()
        numeric_desc = numeric_desc.rename(columns = {'index': 'var'})
        numeric_desc = numeric_desc.sort_values(by = 'missing_rate', ascending = False)
        
    if df_string.shape[0] == 0:
        print ('字符型变量个数为0，不做统计分析')
    else:
        string_cols = list(df_string['var'])
        string_desc = pd.DataFrame()
        for col in string_cols:
            df1 = df.loc[df[col].isin(string_missing_value), [col]]
            df2 = df.loc[~df[col].isin(string_missing_value), [col]]
            
            string = pd.DataFrame()
            string['unique'] = [len(set(df2[col]))]
            string['missing'] = df1.shape[0]
            string['count'] = df.shape[0]
            string['missing_rate'] = string['missing'] *1.0 / string['count']
            string['var'] = col
            string = string[['var', 'count', 'missing', 'missing_rate', 'unique']]
            
            string_desc = pd.concat([string_desc, string])
    
    if df_numeric.shape[0] == 0 and df_string.shape[0] == 0:
        print ('无输出')
    if df_numeric.shape[0] == 0 and df_string.shape[0] != 0:
        return {'string_desc':string_desc}
    if df_numeric.shape[0] != 0 and df_string.shape[0] == 0:
        return {'numeric_desc':numeric_desc}
    if df_numeric.shape[0] != 0 and df_string.shape[0] != 0:
        return {'numeric_desc':numeric_desc, 'string_desc':string_desc}

In [ ]:
'''
Author: error: error: git config user.name & please set dead value or install git && error: git config user.email & please set dead value or install git & please set dead value or install git
Date: 2025-05-16 10:33:15
LastEditors: error: error: git config user.name & please set dead value or install git && error: git config user.email & please set dead value or install git & please set dead value or install git
LastEditTime: 2025-08-20 11:30:53
FilePath: \Project_Business\风控\2.model_func.ipynb
Description: 这是默认设置,请设置`customMade`, 打开koroFileHeader查看配置 进行设置: https://github.com/OBKoro1/koro1FileHeader/wiki/%E9%85%8D%E7%BD%AE
'''
def SplitData(df, col, numOfSplit, special_attribute=[]): 
    ''' 
    :param df: 按照col排序后的数据集 
    :param col: 待分箱的变量 
    :param numOfSplit: 切分的组别数 
    :param special_attribute: 在切分数据集的时候，某些特殊值需要排除在外 
    :return: 在原数据集上增加一列，把原始细粒度的col重新划分成粗粒度的值，便于分箱中的合并处理 
    ''' 
    # 创建数据集的副本，避免修改原始数据 
    df2 = df.copy() 
    if special_attribute != []: 
        df2 = df.loc[~df[col].isin(special_attribute)] 
    N = df2.shape[0] 
    n = int(N/numOfSplit) 
    splitPointIndex = [i*n for i in range(1,numOfSplit)] 
    rawValues = sorted(list(df2[col])) 
    splitPoint = [rawValues[i] for i in splitPointIndex] 
    splitPoint = sorted(list(set(splitPoint))) 
    return splitPoint

In [5]:
def AssignGroup(x, bin):
    '''
    :param x: 某个变量的某个取值
    :param bin: 上述变量的分箱结果
    :return: x在分箱结果下的映射
    '''
    N = len(bin)
    if x<=min(bin):
        return min(bin)
    elif x>max(bin):
        return 10e10 #note
    else:
        for i in range(N-1):
            if bin[i] < x <= bin[i+1]:
                return bin[i+1]

In [6]:
def BinBadRate(df, col, target, grantRateIndicator=0):
    '''
    :param df: 需要计算好坏比率的数据集
    :param col: 需要计算好坏比率的特征
    :param target: 好坏标签
    :param grantRateIndicator: 1返回总体的坏样本率，0不返回
    :return: 每箱的坏样本率，以及总体的坏样本率（当grantRateIndicator＝＝1时）
    '''
    total = df.groupby([col])[target].count()
    total = pd.DataFrame({'total': total})
    bad = df.groupby([col])[target].sum()
    bad = pd.DataFrame({'bad': bad})
    regroup = total.merge(bad, left_index=True, right_index=True, how='left')
#     regroup.reset_index(level=0, inplace=True)
    regroup.reset_index(level = [0,], inplace=True)
    regroup['bad_rate'] = regroup.apply(lambda x: x.bad * 1.0 / x.total, axis=1)
    dicts = dict(zip(regroup[col],regroup['bad_rate']))
    if grantRateIndicator==0:
        return (dicts, regroup)
    N = sum(regroup['total'])
    B = sum(regroup['bad'])
    overallRate = B * 1.0 / N
    return (dicts, regroup, overallRate)

In [7]:
def MergeBad0(df,col,target, direction='bad'):
    '''
     :param df: 包含检验0％或者100%坏样本率
     :param col: 分箱后的变量或者类别型变量。检验其中是否有一组或者多组没有坏样本或者没有好样本。如果是，则需要进行合并
     :param target: 目标变量，0、1表示好、坏
     :return: 合并方案，使得每个组里同时包含好坏样本
     '''
    regroup = BinBadRate(df, col, target)[1]
    if direction == 'bad':
        # 如果是合并0坏样本率的组，则跟最小的非0坏样本率的组进行合并
        regroup = regroup.sort_values(by  = 'bad_rate')
    else:
        # 如果是合并0好样本样本率的组，则跟最小的非0好样本率的组进行合并
        regroup = regroup.sort_values(by = 'bad_rate',ascending = False)
    regroup.index = range(regroup.shape[0])
    col_regroup = [[i] for i in regroup[col]]
    del_index = []
    for i in range(regroup.shape[0]-1):
        col_regroup[i+1] = col_regroup[i] + col_regroup[i+1]
        del_index.append(i)
        if direction == 'bad':
            if regroup['bad_rate'][i+1] > 0:
                break
        else:
            if regroup['bad_rate'][i+1] < 1:
                break
    col_regroup2 = [col_regroup[i] for i in range(len(col_regroup)) if i not in del_index]
    newGroup = {}
    for i in range(len(col_regroup2)):
        for g2 in col_regroup2[i]:
            newGroup[g2] = 'Bin '+str(i)
    return newGroup

In [8]:
def BadRateEncoding(df, col, target):
    '''
    :param df: dataframe containing feature and target
    :param col: the feature that needs to be encoded with bad rate, usually categorical type
    :param target: good/bad indicator
    :return: the assigned bad rate to encode the categorical feature
    '''
    regroup = BinBadRate(df, col, target, grantRateIndicator=0)[1]
    br_dict = regroup[[col,'bad_rate']].set_index([col]).to_dict(orient='index')
    for k, v in br_dict.items():
        br_dict[k] = v['bad_rate']
    badRateEnconding = df[col].map(lambda x: br_dict[x])  #做成字典，逐一遍历映射
    return {'encoding':badRateEnconding, 'bad_rate':br_dict}

In [9]:
def Chi2(df, total_col, bad_col):
    '''
    :param df: 包含全部样本总计与坏样本总计的数据框
    :param total_col: 全部样本的个数
    :param bad_col: 坏样本的个数
    :return: 卡方值
    '''
    df2 = df.copy()
    # 求出df中，总体的坏样本率和好样本率
    badRate = sum(df2[bad_col])*1.0/sum(df2[total_col])
    df2['good'] = df2.apply(lambda x: x[total_col] - x[bad_col], axis = 1)
    goodRate = sum(df2['good']) * 1.0 / sum(df2[total_col])
    # 期望坏（好）样本个数＝全部样本个数*平均坏（好）样本占比
    df2['badExpected'] = df[total_col].apply(lambda x: x*badRate)
    df2['goodExpected'] = df[total_col].apply(lambda x: x * goodRate)
    badCombined = zip(df2['badExpected'], df2[bad_col])
    goodCombined = zip(df2['goodExpected'], df2['good'])
    badChi = [(i[0]-i[1])**2/(i[0] + 0.000001) for i in badCombined]
    goodChi = [(i[0] - i[1]) ** 2 / (i[0] + 0.000001) for i in goodCombined]
    chi2 = sum(badChi) + sum(goodChi)
    return chi2

In [ ]:
def AssignBin(x, cutOffPoints,special_attribute=[]):
    '''
    :param x: 某个变量的某个取值
    :param cutOffPoints: 上述变量的分箱结果，用切分点表示
    :param special_attribute:  不参与分箱的特殊取值
    :return: 分箱后的对应的第几个箱，从0开始
    for example, if cutOffPoints = [10,20,30], if x = 7, return Bin 0. If x = 35, return Bin 3
    '''
    numBin = len(cutOffPoints) + 1
    if x in special_attribute:
        i = special_attribute.index(x) + 1
        return 'Bin {}'.format(0-i)
    if x <= cutOffPoints[0]:
        return 'Bin 0'
    elif x > cutOffPoints[-1]:
        return 'Bin {}'.format(numBin-1)
    else:
        for i in range(0,numBin-1):
            if cutOffPoints[i] < x <=  cutOffPoints[i+1]:
                return 'Bin {}'.format(i+1)

In [11]:
def AssignNum(x, cutOffPoints):
    '''
    :param x: 某个变量的某个取值
    :param cutOffPoints: 上述变量的分箱结果，用切分点表示
    :return: 分箱后的对应的第几个箱，从0开始
    '''
    numBin = len(cutOffPoints) + 1
    if x >= cutOffPoints[0] and x <= cutOffPoints[1]:
        return '{}'.format(0)
    elif x > cutOffPoints[-1]:
        return '{}'.format(numBin-2)
    else:
        for i in range(1,numBin-2):
            if cutOffPoints[i] < x <=  cutOffPoints[i+1]:
                return '{}'.format(i)

In [1]:
def AssignNum_0(x, cutOffPoints):
    '''
    :0单独为一组
    :param x: 某个变量的某个取值
    :param cutOffPoints: 上述变量的分箱结果，用切分点表示
    :return: 分箱后的对应的第几个箱，从0开始
    '''
    numBin = len(cutOffPoints) + 1
    if x <= cutOffPoints[0]:
        return '{}'.format(0)
    elif x > cutOffPoints[-1]:
        return '{}'.format(numBin-1)
    else:
        for i in range(0,numBin-1):
            if cutOffPoints[i] < x <=  cutOffPoints[i+1]:
                return '{}'.format(i + 1)

In [ ]:
def AssignBin_me(x, cutOffPoints,special_attribute=[]):
    '''
    :param x: 某个变量的某个取值
    :param cutOffPoints: 上述变量的分箱结果，用切分点表示
    :param special_attribute:  不参与分箱的特殊取值
    :return: 分箱后的对应的第几个箱，从0开始
    for example, if cutOffPoints = [10,20,30], if x = 7, return Bin 0. If x = 35, return Bin 3
    '''
    numBin = len(cutOffPoints) + 1
    if x in special_attribute:
        i = special_attribute.index(x) + 1
        return '[{},{}]'.format(int(x), int(x))   #注意返回值
    if x <= cutOffPoints[0] and x >= 0:
        return '[0,{}]'.format(cutOffPoints[0])
    elif x > cutOffPoints[-1]:
        return '({},inf)'.format(cutOffPoints[-1])
    else:
        for i in range(0,numBin-1):
            if cutOffPoints[i] < x <=  cutOffPoints[i+1]:
                return '({},{}]'.format(cutOffPoints[i],cutOffPoints[i+1])

In [40]:
def ChiMerge(df, col, target, max_interval=5,special_attribute=[],minBinPcnt=0):
    '''
    :param df: 包含目标变量与分箱属性的数据框
    :param col: 需要分箱的属性
    :param target: 目标变量，取值0或1
    :param max_interval: 最大分箱数。如果原始属性的取值个数低于该参数，不执行这段函数
    :param special_attribute: 不参与分箱的属性取值
    :param minBinPcnt：最小箱的占比，默认为0
    :return: 分箱结果
    '''
    colLevels = sorted(list(set(df[col])))
    N_distinct = len(colLevels)
    if len(special_attribute)>= 1: 
        df1 = df.loc[df[col].isin(special_attribute)]
        df2 = df.loc[~df[col].isin(special_attribute)]
    else:
        df2 = df.copy()
    N_distinct = len(list(set(df2[col])))
    if N_distinct > 100:
        split_x = SplitData(df2, col, 100)
        df2['temp'] = df2[col].map(lambda x: AssignGroup(x, split_x))
    else:
        df2['temp'] = df2[col]
        # 计算每个分组的坏样本率，并获取总体的坏样本率。
    (binBadRate, regroup, overallRate) = BinBadRate(df2, 'temp', target, grantRateIndicator=1)
    # 首先，每个单独的属性值将被分为单独的一组
    # 对属性值进行排序，然后两两组别进行合并
    colLevels = sorted(list(set(df2['temp'])))
    groupIntervals = [[i] for i in colLevels]
    # 步骤一：建立循环，不断合并最优的相邻两个组别，直到 最终分裂出来的分箱数<＝预设的最大分箱数
    if N_distinct > max_interval:
#         if len(special_attribute) == 2 and df1.shape[0] > 0:  # note
        if len(special_attribute) == 4 and df1.shape[0] > 0:  # note
#             split_intervals = max_interval - len(special_attribute) + 1 #  note  
            split_intervals = max_interval - len(special_attribute) + 3 #  note  
        else:
            split_intervals = max_interval - len(special_attribute)
        
        while (len(groupIntervals) > split_intervals):  # 终止条件: 当前分箱数＝预设的分箱数
            # 每次循环时, 计算合并相邻组别后的卡方值。具有最小卡方值的合并方案，是最优方案
            chisqList = []
            for k in range(len(groupIntervals)-1):
                temp_group = groupIntervals[k] + groupIntervals[k+1]
                df2b = regroup.loc[regroup['temp'].isin(temp_group)]
                #chisq = Chi2(df2b, 'total', 'bad', overallRate)
                chisq = Chi2(df2b, 'total', 'bad')
                chisqList.append(chisq)
            best_comnbined = chisqList.index(min(chisqList))
            groupIntervals[best_comnbined] = groupIntervals[best_comnbined] + groupIntervals[best_comnbined+1]
            # after combining two intervals, we need to remove one of them
            groupIntervals.remove(groupIntervals[best_comnbined + 1])
        groupIntervals = [sorted(i) for i in groupIntervals]
        cutOffPoints = [max(i) for i in groupIntervals[:-1]]
    else:
        cutOffPoints = colLevels[:-1]
    #步骤二：检查是否有箱没有好或者坏样本。如果有，需要跟相邻的箱进行合并，直到每箱同时包含好坏样本
    groupedvalues = df2['temp'].apply(lambda x: AssignBin(x, cutOffPoints))
    df2['temp_Bin'] = groupedvalues
    (binBadRate,regroup) = BinBadRate(df2, 'temp_Bin', target)
    groupedvalues = df2['temp'].apply(lambda x: AssignBin(x, cutOffPoints))
    df2['temp_Bin'] = groupedvalues
    [minBadRate, maxBadRate] = [min(binBadRate.values()),max(binBadRate.values())]
    while minBadRate ==0 or maxBadRate == 1:
        # 找出全部为好／坏样本的箱
        indexForBad01 = regroup[regroup['bad_rate'].isin([0,1])].temp_Bin.tolist()
        bin=indexForBad01[0]
        #以下这种情况自己写(原来就只有1个切分点的情况)
        if  len(cutOffPoints) == 1 and bin == max(regroup.temp_Bin):
            cutOffPoints = [df2['temp'].max()]
            groupedvalues = df2['temp'].apply(lambda x: AssignBin(x, cutOffPoints))
            df2['temp_Bin'] = groupedvalues
            (binBadRate, regroup) = BinBadRate(df2, 'temp_Bin', 'y')
            [minBadRate, maxBadRate] = [min(binBadRate.values()), max(binBadRate.values())]
            break
        #以上这种情况自己写
        # 如果是最后一箱，则需要和上一个箱进行合并，也就意味着分裂点cutOffPoints中的最后一个需要移除
        if bin == max(regroup.temp_Bin):
            cutOffPoints = cutOffPoints[:-1]
        # 如果是第一箱，则需要和下一个箱进行合并，也就意味着分裂点cutOffPoints中的第一个需要移除
        elif bin == min(regroup.temp_Bin):
            cutOffPoints = cutOffPoints[1:]
        # 如果是中间的某一箱，则需要和前后中的一个箱进行合并，依据是较小的卡方值
        else:
            # 和前一箱进行合并，并且计算卡方值
            currentIndex = list(regroup.temp_Bin).index(bin)
            prevIndex = list(regroup.temp_Bin)[currentIndex - 1]
            df3 = df2.loc[df2['temp_Bin'].isin([prevIndex, bin])]
            (binBadRate, df2b) = BinBadRate(df3, 'temp_Bin', target)
            #chisq1 = Chi2(df2b, 'total', 'bad', overallRate)
            chisq1 = Chi2(df2b, 'total', 'bad')
            # 和后一箱进行合并，并且计算卡方值
            laterIndex = list(regroup.temp_Bin)[currentIndex + 1]
            df3b = df2.loc[df2['temp_Bin'].isin([laterIndex, bin])]
            (binBadRate, df2b) = BinBadRate(df3b, 'temp_Bin', target)
            #chisq2 = Chi2(df2b, 'total', 'bad', overallRate)
            chisq2 = Chi2(df2b, 'total', 'bad')
            if chisq1 < chisq2:
                cutOffPoints.remove(cutOffPoints[currentIndex - 1])
            else:
                cutOffPoints.remove(cutOffPoints[currentIndex])
        # 完成合并之后，需要再次计算新的分箱准则下，每箱是否同时包含好坏样本
        groupedvalues = df2['temp'].apply(lambda x: AssignBin(x, cutOffPoints))     
        df2['temp_Bin'] = groupedvalues
        (binBadRate, regroup) = BinBadRate(df2, 'temp_Bin', target)
        [minBadRate, maxBadRate] = [min(binBadRate.values()), max(binBadRate.values())]
        #以下这种情况自己写(合并到最后只有1个切分点的情况)
        if  len(cutOffPoints) == 1 and len(indexForBad01) > 1:
            cutOffPoints = [df2['temp'].max()]
            groupedvalues = df2['temp'].apply(lambda x: AssignBin(x, cutOffPoints))
            df2['temp_Bin'] = groupedvalues
            (binBadRate, regroup) = BinBadRate(df2, 'temp_Bin', 'y')
            [minBadRate, maxBadRate] = [min(binBadRate.values()), max(binBadRate.values())]
            break
        #以上这种情况自己写  
    #步骤三：每箱的占比不低于预设值（可选）
    # 如果有特殊属性，那么最终分裂出来的分箱数＝预设的最大分箱数－特殊属性的个数
    # 需要检查分箱后的最小占比
    if minBinPcnt > 0:
        groupedvalues = df2['temp'].apply(lambda x: AssignBin(x, cutOffPoints))
        df2['temp_Bin'] = groupedvalues
        valueCounts = groupedvalues.value_counts().to_frame()
        valueCounts['pcnt'] = valueCounts['temp'].apply(lambda x: x * 1.0 / N)
        valueCounts = valueCounts.sort_index()
        minPcnt = min(valueCounts['pcnt'])
        while minPcnt < minBinPcnt and len(cutOffPoints) > 2:
            # 找出占比最小的箱
            indexForMinPcnt = valueCounts[valueCounts['pcnt'] == minPcnt].index.tolist()[0]
            # 如果占比最小的箱是最后一箱，则需要和上一个箱进行合并，也就意味着分裂点cutOffPoints中的最后一个需要移除
            if indexForMinPcnt == max(valueCounts.index):
                cutOffPoints = cutOffPoints[:-1]
            # 如果占比最小的箱是第一箱，则需要和下一个箱进行合并，也就意味着分裂点cutOffPoints中的第一个需要移除
            elif indexForMinPcnt == min(valueCounts.index):
                cutOffPoints = cutOffPoints[1:]
            # 如果占比最小的箱是中间的某一箱，则需要和前后中的一个箱进行合并，依据是较小的卡方值
            else:
                # 和前一箱进行合并，并且计算卡方值
                currentIndex = list(valueCounts.index).index(indexForMinPcnt)
                prevIndex = list(valueCounts.index)[currentIndex - 1]
                df3 = df2.loc[df2['temp_Bin'].isin([prevIndex, indexForMinPcnt])]
                (binBadRate, df2b) = BinBadRate(df3, 'temp_Bin', target)
                #chisq1 = Chi2(df2b, 'total', 'bad', overallRate)
                chisq1 = Chi2(df2b, 'total', 'bad')
                # 和后一箱进行合并，并且计算卡方值
                laterIndex = list(valueCounts.index)[currentIndex + 1]
                df3b = df2.loc[df2['temp_Bin'].isin([laterIndex, indexForMinPcnt])]
                (binBadRate, df2b) = BinBadRate(df3b, 'temp_Bin', target)
                #chisq2 = Chi2(df2b, 'total', 'bad', overallRate)
                chisq2 = Chi2(df2b, 'total', 'bad')
                if chisq1 < chisq2:
                    cutOffPoints.remove(cutOffPoints[currentIndex - 1])
                else:
                    cutOffPoints.remove(cutOffPoints[currentIndex])
    cutOffPoints = special_attribute + cutOffPoints
    return cutOffPoints

In [ ]:
# 改过的ChiMerge;
def ChiMerge(df, col, target, max_interval=5, special_attribute=[], minBinPcnt=0):
    colLevels = sorted(list(set(df[col])))
    N_distinct = len(colLevels)
    if len(special_attribute) >= 1:
        df1 = df.loc[df[col].isin(special_attribute)]
        df2 = df.loc[~df[col].isin(special_attribute)]
    else:
        df2 = df.copy()
    N_distinct = len(list(set(df2[col])))
    if N_distinct > 50:
        split_x = SplitData(df2, col, 50)
        df2['temp'] = df2[col].map(lambda x: AssignGroup(x, split_x))
    else:
        df2['temp'] = df2[col]
    (binBadRate, regroup, overallRate) = BinBadRate(df2, 'temp', target, grantRateIndicator=1)

    colLevels = sorted(list(set(df2['temp'])))
    groupIntervals = [[i] for i in colLevels]

    if N_distinct > max_interval:
        if len(special_attribute) == 2 and df1.shape[0] > 0:  # note
            split_intervals = max_interval - len(special_attribute) + 1  # note
        else:
            split_intervals = max_interval - len(special_attribute)

        while (len(groupIntervals) > split_intervals):  # 终止条件: 当前分箱数＝预设的分箱数

            chisqList = []
            for k in range(len(groupIntervals) - 1):
                temp_group = groupIntervals[k] + groupIntervals[k + 1]
                df2b = regroup.loc[regroup['temp'].isin(temp_group)]
                # chisq = Chi2(df2b, 'total', 'bad', overallRate)
                chisq = Chi2(df2b, 'total', 'bad')
                chisqList.append(chisq)
            best_comnbined = chisqList.index(min(chisqList))
            groupIntervals[best_comnbined] = groupIntervals[best_comnbined] + groupIntervals[best_comnbined + 1]
            # after combining two intervals, we need to remove one of them
            groupIntervals.remove(groupIntervals[best_comnbined + 1])
        groupIntervals = [sorted(i) for i in groupIntervals]
        cutOffPoints = [max(i) for i in groupIntervals[:-1]]
    else:
        cutOffPoints = colLevels[:-1]

    groupedvalues = df2['temp'].apply(lambda x: AssignBin(x, cutOffPoints))
    df2['temp_Bin'] = groupedvalues
    (binBadRate, regroup) = BinBadRate(df2, 'temp_Bin', target)
    groupedvalues = df2['temp'].apply(lambda x: AssignBin(x, cutOffPoints))
    df2['temp_Bin'] = groupedvalues
    [minBadRate, maxBadRate] = [min(binBadRate.values()), max(binBadRate.values())]
    while minBadRate == 0 or maxBadRate == 1:
        # 找出全部为好／坏样本的箱
        indexForBad01 = regroup[regroup['bad_rate'].isin([0, 1])].temp_Bin.tolist()
        bin = indexForBad01[0]

        if len(cutOffPoints) == 1 and bin == max(regroup.temp_Bin):
            cutOffPoints = [df2['temp'].max()]
            groupedvalues = df2['temp'].apply(lambda x: AssignBin(x, cutOffPoints))
            df2['temp_Bin'] = groupedvalues
            (binBadRate, regroup) = BinBadRate(df2, 'temp_Bin', target)
            [minBadRate, maxBadRate] = [min(binBadRate.values()), max(binBadRate.values())]
            break

        if bin == max(regroup.temp_Bin):
            cutOffPoints = cutOffPoints[:-1]

        elif bin == min(regroup.temp_Bin):
            cutOffPoints = cutOffPoints[1:]

        else:

            currentIndex = list(regroup.temp_Bin).index(bin)
            prevIndex = list(regroup.temp_Bin)[currentIndex - 1]
            df3 = df2.loc[df2['temp_Bin'].isin([prevIndex, bin])]
            (binBadRate, df2b) = BinBadRate(df3, 'temp_Bin', target)
            # chisq1 = Chi2(df2b, 'total', 'bad', overallRate)
            chisq1 = Chi2(df2b, 'total', 'bad')

            laterIndex = list(regroup.temp_Bin)[currentIndex + 1]
            df3b = df2.loc[df2['temp_Bin'].isin([laterIndex, bin])]
            (binBadRate, df2b) = BinBadRate(df3b, 'temp_Bin', target)
            # chisq2 = Chi2(df2b, 'total', 'bad', overallRate)
            chisq2 = Chi2(df2b, 'total', 'bad')
            if chisq1 < chisq2:
                cutOffPoints.remove(cutOffPoints[currentIndex - 1])
            else:
                cutOffPoints.remove(cutOffPoints[currentIndex])

        groupedvalues = df2['temp'].apply(lambda x: AssignBin(x, cutOffPoints))
        df2['temp_Bin'] = groupedvalues
        (binBadRate, regroup) = BinBadRate(df2, 'temp_Bin', target)
        [minBadRate, maxBadRate] = [min(binBadRate.values()), max(binBadRate.values())]

        if len(cutOffPoints) == 1 and len(indexForBad01) > 1:
            cutOffPoints = [df2['temp'].max()]
            groupedvalues = df2['temp'].apply(lambda x: AssignBin(x, cutOffPoints))
            df2['temp_Bin'] = groupedvalues
            (binBadRate, regroup) = BinBadRate(df2, 'temp_Bin', target)
            [minBadRate, maxBadRate] = [min(binBadRate.values()), max(binBadRate.values())]
            break

    if minBinPcnt > 0:
        groupedvalues = df2['temp'].apply(lambda x: AssignBin(x, cutOffPoints))
        df2['temp_Bin'] = groupedvalues
        valueCounts = groupedvalues.value_counts().to_frame()
        valueCounts['pcnt'] = valueCounts['temp'].apply(lambda x: x * 1.0 / N)
        valueCounts = valueCounts.sort_index()
        minPcnt = min(valueCounts['pcnt'])
        while minPcnt < minBinPcnt and len(cutOffPoints) > 2:

            indexForMinPcnt = valueCounts[valueCounts['pcnt'] == minPcnt].index.tolist()[0]

            if indexForMinPcnt == max(valueCounts.index):
                cutOffPoints = cutOffPoints[:-1]

            elif indexForMinPcnt == min(valueCounts.index):
                cutOffPoints = cutOffPoints[1:]

            else:

                currentIndex = list(valueCounts.index).index(indexForMinPcnt)
                prevIndex = list(valueCounts.index)[currentIndex - 1]
                df3 = df2.loc[df2['temp_Bin'].isin([prevIndex, indexForMinPcnt])]
                (binBadRate, df2b) = BinBadRate(df3, 'temp_Bin', target)
                # chisq1 = Chi2(df2b, 'total', 'bad', overallRate)
                chisq1 = Chi2(df2b, 'total', 'bad')

                laterIndex = list(valueCounts.index)[currentIndex + 1]
                df3b = df2.loc[df2['temp_Bin'].isin([laterIndex, indexForMinPcnt])]
                (binBadRate, df2b) = BinBadRate(df3b, 'temp_Bin', target)
                # chisq2 = Chi2(df2b, 'total', 'bad', overallRate)
                chisq2 = Chi2(df2b, 'total', 'bad')
                if chisq1 < chisq2:
                    cutOffPoints.remove(cutOffPoints[currentIndex - 1])
                else:
                    cutOffPoints.remove(cutOffPoints[currentIndex])
    cutOffPoints = special_attribute + cutOffPoints
    return cutOffPoints

In [13]:
## 判断某变量的坏样本率是否单调
def BadRateMonotone(df, sortByVar, target,special_attribute = []):
    '''
    :param df: 包含检验坏样本率的变量，和目标变量
    :param sortByVar: 需要检验坏样本率的变量
    :param target: 目标变量，0、1表示好、坏
    :param special_attribute: 不参与检验的特殊值
    :return: 坏样本率单调与否
    '''
    df2 = df.loc[~df[sortByVar].isin(special_attribute)]
    if len(set(df2[sortByVar])) <= 2:
        return True
    regroup = BinBadRate(df2, sortByVar, target)[1]
    combined = zip(regroup['total'],regroup['bad'])
    badRate = [x[1]*1.0/x[0] for x in combined]
    badRateNotMonotone = [badRate[i]<badRate[i+1] and badRate[i] < badRate[i-1] or badRate[i]>badRate[i+1] and badRate[i] > badRate[i-1]
                       for i in range(1,len(badRate)-1)]
    if True in badRateNotMonotone:
        return False
    else:
        return True

In [9]:
def CalcWOE(df, col, target):
    '''
    :param df: 包含需要计算WOE的变量和目标变量
    :param col: 需要计算WOE、IV的变量，必须是分箱后的变量，或者不需要分箱的类别型变量
    :param target: 目标变量，0、1表示好、坏
    :return: 返回WOE和IV、变量每个分箱的明细
    '''
    total = df.groupby([col])[target].count()
    total = pd.DataFrame({'total': total})
    bad = df.groupby([col])[target].sum()
    bad = pd.DataFrame({'bad': bad})
    regroup = total.merge(bad, left_index=True, right_index=True, how='left')
    regroup.reset_index(level=0, inplace=True)
    N = sum(regroup['total'])
    B = sum(regroup['bad'])
    regroup['bad'] = regroup['bad'].map(lambda x: 0.5 if x == 0 else x)  #平滑
    regroup['good'] = regroup['total'] - regroup['bad']
    regroup['good'] = regroup['good'].map(lambda x: 0.5 if x == 0 else x)  #平滑
    G = N - B
    regroup['bad_pcnt'] = regroup['bad'].map(lambda x: x*1.0/B)
    regroup['good_pcnt'] = regroup['good'].map(lambda x: x * 1.0 / G)
    regroup['WOE'] = regroup.apply(lambda x: np.log(x.good_pcnt*1.0/x.bad_pcnt),axis = 1)
    regroup['IV'] = regroup.apply(lambda x: (x.good_pcnt-x.bad_pcnt)*np.log(x.good_pcnt*1.0/x.bad_pcnt),axis = 1)
    regroup['var'] = col
    regroup['bad_rate'] = regroup['bad']*1.0 / regroup['total']
    regroup['group_rate'] = regroup['total']*1.0 / N
    
    WOE_dict = regroup[[col,'WOE']].set_index(col).to_dict(orient='index')
    for k, v in WOE_dict.items():
        WOE_dict[k] = v['WOE']
    IV = regroup.apply(lambda x: (x.good_pcnt-x.bad_pcnt)*np.log(x.good_pcnt*1.0/x.bad_pcnt),axis = 1)
    IV = sum(IV)
    return {"WOE": WOE_dict, 'IV':IV, 'regroup':regroup}

In [10]:
def var_colinearity(df, high_IV_sorted, type_, correlation):
    '''
    df：需要分析数据集
    high_IV_sorted：需要分析的变量及iv，list
    iv_var_WOE：保存删去 相关系数大且IV大 的结果
    '''
    deleted_index = []
    cnt_vars = len(high_IV_sorted)
    #以下循环删去相关系数大的
    for i in range(cnt_vars):
        if i in deleted_index:
            continue
        x1 = high_IV_sorted[i][0]+"_WOE"
        for j in range(cnt_vars):
            if i == j or j in deleted_index:
                continue
            y1 = high_IV_sorted[j][0]+"_WOE"
            roh = np.corrcoef(df[x1],df[y1])[0,1]
            if abs(roh) > correlation:                   #阈值确定
                x1_IV = high_IV_sorted[i][1]
                y1_IV = high_IV_sorted[j][1]
                if x1_IV > y1_IV:
                    deleted_index.append(j)
                else :
                    deleted_index.append(i)
    iv_var_WOE = [high_IV_sorted[i][0]+"_WOE" for i in range(cnt_vars) if i not in deleted_index]
    iv_var_Bin = [high_IV_sorted[i][0] for i in range(cnt_vars) if i not in deleted_index]
    
    #以下跑 VIF，变量个数 >= 2
    if len(iv_var_WOE) >= 2:
        X = np.matrix(df[iv_var_WOE])
        VIF_list = [variance_inflation_factor(X, i) for i in range(X.shape[1])]
        max_VIF = max(VIF_list)
        print ('{} 的 max_VIF :{}'.format(type_, max_VIF))
    else:
        print('{}：只有一个变量'.format(type_))
    return {'iv_var_WOE':iv_var_WOE, 'iv_var_Bin':iv_var_Bin}

In [3]:
def var_vif(df, cols, target):
    '''
    df：数据集
    cols：求vif的变量，list
    target：因变量
    '''
    features = " + ".join(cols)
    use_df = df[cols + [target]]
    y, x = dmatrices(target + ' ~ ' + features, use_df, return_type='dataframe')
    data = pd.DataFrame()
    data.loc[:, 'var'] = x.columns
    data.loc[:, 'ols_vif'] = [variance_inflation_factor(x.values, i) for i in range(x.shape[1])]
    return data

In [20]:
def KS(df, score, target):
    '''
    :param df: 包含目标变量与预测值的数据集
    :param score: 得分或者概率
    :param target: 目标变量
    :return: KS值
    '''
    total = df.groupby([score])[target].count()
    bad = df.groupby([score])[target].sum()
    all = pd.DataFrame({'total':total, 'bad':bad})
    all['good'] = all['total'] - all['bad']
    all[score] = all.index
    all = all.sort_values(by=score,ascending=False)
    all.index = range(len(all))
    all['badCumRate'] = all['bad'].cumsum() / all['bad'].sum()
    all['goodCumRate'] = all['good'].cumsum() / all['good'].sum()
    KS = all.apply(lambda x: x.badCumRate - x.goodCumRate, axis=1)
    return max(KS)

In [1]:
def sm_Logit(train, test, all_var_WOE, pvals_threshold = 0.1):
    '''
    train：训练集
    test：测试集
    all_var_WOE：入模变量
    pvals_threshold：显著性阈值
    return：返回模型变量和模型变量对应的显著性、IV值、中文解释的表
    
    '''
    X_train = train[all_var_WOE] 
    y_train = train['y']
    
    X_test = test[all_var_WOE] 
    y_test = test['y']
    
    lr_var = all_var_WOE[:]
    #模型训练
    X_train['intercept'] = [1]*X_train.shape[0]
    LR = sm.Logit(y_train, X_train).fit()
    
    #模型调整
    pvals = LR.pvalues
    pvals = pvals.to_dict()
    varLargeP = {k: v for k,v in pvals.items() if v >= pvals_threshold}  #注意阈值
    varLargeP = sorted(varLargeP.items(), key=lambda d:d[1], reverse = True)
    
    params = LR.params
    params = params.to_dict()
    params = {k: v for k,v in params.items() if v > 0}
    params = sorted(params.items(), key=lambda d:d[1], reverse = True)
    
    varLargeP = varLargeP + params
    
    while(len(varLargeP) > 0 and len(lr_var) > 0):
        varMaxP = varLargeP[0][0]
        if varMaxP == 'intercept':
            print ('the intercept is not significant!')
            break
        lr_var.remove(varMaxP)
        X_train = X_train[lr_var]
        X_train['intercept'] = [1] * X_train.shape[0]
        LR = sm.Logit(y_train, X_train).fit()
        
        pvals = LR.pvalues
        pvals = pvals.to_dict()
        varLargeP = {k: v for k,v in pvals.items() if v >= pvals_threshold}  #注意阈值
        varLargeP = sorted(varLargeP.items(), key=lambda d:d[1], reverse = True)
    
        params = LR.params
        params = params.to_dict()
        params = {k: v for k,v in params.items() if v > 0}
        params = sorted(params.items(), key=lambda d:d[1], reverse = True)
    
        varLargeP = varLargeP + params
        
    LR_params = list(LR.params.index)
    LR_params.remove('intercept')
    summary = LR.summary()
    print (summary)
    
    pvals = LR.pvalues
    pvals = pvals.to_dict()
    pvals_pd = pd.DataFrame()
    pvals_pd['var'] = pvals.keys()
    pvals_pd['p-value'] = pvals.values()
    
    params = LR.params
    params = params.to_dict()
    pvals_pd['coef'] = params.values()
    
    pvals_pd = pvals_pd.merge(iv_dex_pool, on = 'var', how = 'left')
    pvals_pd = pvals_pd.drop(['long_name'], axis = 1)
    pvals_pd = pvals_pd.rename(columns = {'var': 'var_WOE'})
    pvals_pd['var'] = pvals_pd['var_WOE'].map(lambda x: x.replace('_WOE', ''))
    pvals_pd = pvals_pd.sort_values(by = 'p-value', ascending = True)

    y_pred = LR.predict(X_train)
    scorecard_result = pd.DataFrame({'prob':y_pred, 'target':y_train})
    train['y_pred'] = LR.predict(X_train)
    print ('训练集KS：{}'.format(KS(scorecard_result,'prob','target')))
    print ('训练集AUC：{}'.format(roc_auc_score(train['y'], train['y_pred'])))
    
    X_test['intercept'] = [1]*X_test.shape[0]
    X_test = X_test[list(LR.params.index)]
    y_pred = LR.predict(X_test)
    test['y_pred'] = LR.predict(X_test)
    scorecard_result = pd.DataFrame({'prob':y_pred, 'target':y_test})
    print ('测试集KS：{}'.format(KS(scorecard_result,'prob','target')))
    print ('测试集AUC：{}'.format(roc_auc_score(test['y'], test['y_pred'])))
    
    return {"LR_params":LR_params, "pvals_pd":pvals_pd}

In [2]:
def model_var_detail(base_score, PDO, train, lr_params_pd, test, regroup_all):
    
    BaseOdds = (train['y'].value_counts()[0]) / (train['y'].value_counts()[1])
    beta = PDO*1.0/np.log(2)
    alpha = base_score - beta * np.log(BaseOdds)
    model_params = list(lr_params_pd['var_WOE'])
    model_params.remove('intercept')
    regroup_test = pd.DataFrame()
    
    for col in model_params:
        col = col.replace('_WOE', '')
        total = test.groupby([col])['y'].count()
        total = pd.DataFrame({'total': total})
        total.reset_index(level=0, inplace=True)
        total.rename(columns = {col:'Bin'}, inplace=True)
        total['var'] = col
        total['Bin'] = total['Bin'].astype('str')
        
        regroup_test = pd.concat([regroup_test, total])
    
    regroup_all_ = regroup_all[regroup_all['var'].isin(list(lr_params_pd['var']))]
    
    regroup_model_var = regroup_all_.merge(regroup_test, on = ['var', 'Bin'], how = 'outer', suffixes = ('_x', '_y'))
#     regroup_model_var['bad_rate'] = regroup_model_var['bad']*1.0 / regroup_model_var['total_x']
    total_x = train.shape[0]
    total_y = test.shape[0]
    regroup_model_var['psi'] = (regroup_model_var['total_y']*1.0 / total_y - regroup_model_var['total_x']*1.0 / total_x)*np.log((regroup_model_var['total_y']*1.0 / total_y)/(regroup_model_var['total_x']*1.0 / total_x))
    
    #加入每个变量每组的得分
    intercept = list(lr_params_pd[lr_params_pd['var_WOE'] == 'intercept'].loc[:, 'coef'])[0]
    n = lr_params_pd.shape[0] - 1
    
    regroup_model_var = regroup_model_var.merge(lr_params_pd, on = 'var', how = 'inner')
    regroup_model_var['score'] = alpha/n - beta*(intercept/n + regroup_model_var['WOE']*regroup_model_var['coef'])
    regroup_model_var['score'] = regroup_model_var['score'].map(lambda x: int(x))
    regroup_model_var = regroup_model_var[['var', 'explain', 'Bin','bad', 'good', 'bad_rate', 'group_rate', 'bad_pcnt', 'good_pcnt', 'WOE', 'IV', 'total_x', 'total_y', 'psi', 'score']]
    
    return regroup_model_var

In [4]:
def KS_ROC_plot(df, y_variable, y_pred, score, title , plot_text = ''):
    '''
    df：输入的数据集
    y_variable：因变量
    y_pred：预测为1的概率
    score：预测为1的分数
    '''
    print ('{}：plot'.format(title))
    df['1_y'] = 1 - df[y_variable]
    df_grouped = df[[y_pred, y_variable, '1_y']].groupby(y_pred).agg(
        {y_variable: np.sum, '1_y': np.sum}).reset_index()
    
    sensitivity = df_grouped.sort_values(y_pred, ascending = False)
    sensitivity['cum_sum'] = sensitivity[y_variable].cumsum()
    sensitivity = sensitivity.sort_values('cum_sum', ascending = False)['cum_sum'] / np.sum(df[y_variable])
    
    specificity = df_grouped.sort_values(y_pred)
    specificity['cum_sum'] = specificity['1_y'].cumsum()
    specificity = specificity['cum_sum'] / (df.shape[0] - np.sum(df[y_variable]))
    
    auc = roc_auc_score(df['y'],df[y_pred]) 
    
    df_grouped_ks = df[[score, y_variable, '1_y']].groupby(score).agg(
        {y_variable: np.sum, '1_y': np.sum}).reset_index()
    
    df_grouped_ks['cum_pct_0'] = df_grouped_ks['1_y'].cumsum() / (
        df.shape[0] - np.sum(df[y_variable]))
    df_grouped_ks['cum_pct_1'] = df_grouped_ks[y_variable].cumsum() / np.sum(df[y_variable])
    df_grouped_ks['ks'] = (df_grouped_ks['cum_pct_1'] - df_grouped_ks['cum_pct_0']) 
    max_ks = df_grouped_ks.loc[df_grouped_ks['ks'].idxmax() : df_grouped_ks['ks'].idxmax()]
    
    plt.plot(1 - specificity, sensitivity, linewidth = 2)
    plt.plot([0, 1], [0, 1], color = 'grey')
    plt.text(0.7, 0.16, plot_text, fontsize = 14)
    plt.text(0.7, 0.1, 'RoC Curve', fontsize = 14)
    plt.text(0.7, 0.04, 'AUC = %.4f' % auc, fontsize = 14)
    plt.xlabel('1 - Specificity', size = 14)
    plt.ylabel('Sensitivity', size = 14)
    fig = plt.gcf()
    plt.show()
    fig.savefig('{} auc.png'.format(title), dpi = 100)
    
    plt.plot(df_grouped_ks[score], df_grouped_ks['cum_pct_0'], linewidth = 2)
    plt.plot(df_grouped_ks[score], df_grouped_ks['cum_pct_1'], linewidth = 2)
    plt.plot([max_ks[score].iloc[0], max_ks[score].iloc[0]], 
             [max_ks['cum_pct_0'].iloc[0], max_ks['cum_pct_1'].iloc[0]], color = 'grey')
    plt.text(0.1 + 0.4 * np.max(df_grouped_ks[score]), np.min(df_grouped_ks['cum_pct_1']) + 0.16, 
                                plot_text, fontsize = 14)
    
    plt.text(500, 0.1, 'Score. Distribution', fontsize = 14)
    plt.text(500, 0.04, 'KS = %.4f' % max_ks['ks'].iloc[0], fontsize = 14)
    
#     plt.text(0.1 + 0.08 * np.max(df_grouped_ks[score]), np.min(df_grouped_ks['cum_pct_1']) + 0.1, 
#                                 'Score. Distribution', fontsize = 14)
#     plt.text(0.1 + 0.08 * np.max(df_grouped_ks[score]), np.min(df_grouped_ks['cum_pct_1']) + 0.04, 
#                                 'KS = %.4f' % max_ks['ks'].iloc[0], fontsize = 14)
    plt.xlabel('score', size = 14)
    plt.ylabel('Proportion', size = 14)
    fig = plt.gcf()
    plt.show()
    fig.savefig('{} ks.png'.format(title), dpi = 100)
    plt.close()

In [37]:
def model_group_qcut(df1, df, target, score, number_of_groups = 10):
    '''
    :param df_tmp: 包含目标变量与预测值的数据集，一般是训练集
    :param df_tmp1: 需要分组统计的数据集，一般是训练集、验证集、测试集，分组和训练集的标准保持一致
    :param score: 得分或者概率；
    :param target: 目标变量
    '''
    df_tmp1 = df1.copy()
    df_tmp = df.copy()
    
    df_tmp1 = df_tmp1[df_tmp1[target].notnull()]
    df_tmp1 = df_tmp1.copy().reset_index().drop('index', axis = 1)
    
    quantiles = np.linspace(0, 1, number_of_groups + 1)
    cutOffPoints = algos.quantile(df_tmp1[score], quantiles)
    
    df_tmp[(df_tmp[score] > cutOffPoints.max())] = cutOffPoints.max()
    df_tmp[(df_tmp[score] < cutOffPoints.min())] = cutOffPoints.min()
    
    cutOffPoints = sorted(list(set(cutOffPoints)))

    bins = pd.DataFrame(pd.cut(df_tmp[score], cutOffPoints)).rename(columns = {score: 'bin_group'})
        
    df_tmp = df_tmp.merge(bins, how = 'left', left_index = True, right_index = True)
    
    df_tmp['cnt_group_total'] = 1
    df_tmp['cnt_bad'] = df_tmp[target].apply(lambda x: 1 if x == 1 else 0)
    df_tmp['cnt_good'] = df_tmp[target].apply(lambda x: 1 if x == 0 else 0)
    
    df_tmp_grouped = df_tmp.groupby('bin_group').agg({'cnt_bad': np.sum, 'cnt_good': np.sum, 'cnt_group_total':np.sum})
    df_tmp_grouped = df_tmp_grouped.reset_index()
    
    if df_tmp[score].max() > 1:
        df_tmp_grouped = df_tmp_grouped.sort_values(by = 'bin_group', ascending = True)
    else:
        df_tmp_grouped = df_tmp_grouped.sort_values(by = 'bin_group', ascending = False)
        
    df_tmp_grouped['cum_bad'] = df_tmp_grouped['cnt_bad'].cumsum()
    df_tmp_grouped['cum_good'] = df_tmp_grouped['cnt_good'].cumsum()
    B = df_tmp_grouped['cnt_bad'].sum()
    G = df_tmp_grouped['cnt_good'].sum()
    df_tmp_grouped['group_bad_rate'] = df_tmp_grouped['cnt_bad'] / df_tmp_grouped['cnt_group_total']
    df_tmp_grouped['cum_bad_pct'] = df_tmp_grouped['cum_bad'] / B
    df_tmp_grouped['cum_good_pct'] = df_tmp_grouped['cum_good'] / G
    df_tmp_grouped['KS'] = df_tmp_grouped['cum_bad_pct'] - df_tmp_grouped['cum_good_pct']
    df_tmp_grouped['KS'] = df_tmp_grouped['KS'].apply(lambda x: abs(x))
    
    return df_tmp_grouped

In [25]:
def model_group_qcut_score(df1, df, target, score, number_of_groups = 10):
    '''
    :param df_tmp: 包含目标变量与预测值的数据集，一般是训练集
    :param df_tmp1: 需要分组统计的数据集，一般是训练集、验证集、测试集，分组和训练集的标准保持一致
    :param score: 得分或者概率；
    :param target: 目标变量
    '''
    df_tmp1 = df1[[target,score]].copy()
    df_tmp = df.copy()
    
    df_tmp1 = df_tmp1[df_tmp1[target].notnull()]
    score_max = df_tmp1[score].max()
    score_min = df_tmp1[score].min()
    n = (score_max - score_min)/number_of_groups
     
    cutOffPoints = []
    for i in range(number_of_groups + 1):
        bins = i * n + score_min
        cutOffPoints.append(bins)
    cutOffPoints = cutOffPoints[:-1]
    cutOffPoints = cutOffPoints + [score_max]
    
    df_tmp[(df_tmp[score] > score_max)] = score_max
    df_tmp[(df_tmp[score] < score_min)] = score_min
    
    cutOffPoints = sorted(list(set(cutOffPoints)))

    bins = pd.DataFrame(pd.cut(df_tmp[score], cutOffPoints)).rename(columns = {score: 'bin_group'})
        
    df_tmp = df_tmp.merge(bins, how = 'left', left_index = True, right_index = True)
    
    df_tmp['cnt_group_total'] = 1
    df_tmp['cnt_bad'] = df_tmp[target].apply(lambda x: 1 if x == 1 else 0)
    df_tmp['cnt_good'] = df_tmp[target].apply(lambda x: 1 if x == 0 else 0)
    
    df_tmp_grouped = df_tmp.groupby('bin_group').agg({'cnt_bad': np.sum, 'cnt_good': np.sum, 'cnt_group_total':np.sum})
    df_tmp_grouped = df_tmp_grouped.reset_index()
    
#     if df_tmp[score].max() > 1:
#         df_tmp_grouped = df_tmp_grouped.sort_values(by = 'bin_group', ascending = True)
#     else:
#         df_tmp_grouped = df_tmp_grouped.sort_values(by = 'bin_group', ascending = True)
        
    df_tmp_grouped['cum_bad'] = df_tmp_grouped['cnt_bad'].cumsum()
    df_tmp_grouped['cum_good'] = df_tmp_grouped['cnt_good'].cumsum()
    B = df_tmp_grouped['cnt_bad'].sum()
    G = df_tmp_grouped['cnt_good'].sum()
    df_tmp_grouped['group_bad_rate'] = df_tmp_grouped['cnt_bad'] / df_tmp_grouped['cnt_group_total']
    df_tmp_grouped['cum_bad_pct'] = df_tmp_grouped['cum_bad'] / B
    df_tmp_grouped['cum_good_pct'] = df_tmp_grouped['cum_good'] / G
    df_tmp_grouped['KS'] = df_tmp_grouped['cum_bad_pct'] - df_tmp_grouped['cum_good_pct']
    df_tmp_grouped['KS'] = df_tmp_grouped['KS'].apply(lambda x: abs(x))
    
    return df_tmp_grouped

In [ ]:
def var_colinearity_ori(df, high_IV_sorted, type_, correlation):
    '''
    df：需要分析数据集
    high_IV_sorted：需要分析的变量及iv，list
    iv_var_WOE：保存删去 相关系数大且IV大 的结果
    '''
    deleted_index = []
    cnt_vars = len(high_IV_sorted)
    #以下循环删去相关系数大的
    for i in range(cnt_vars):
        if i in deleted_index:
            continue
        x1 = high_IV_sorted[i][0]
        for j in range(cnt_vars):
            if i == j or j in deleted_index:
                continue
            y1 = high_IV_sorted[j][0]
            roh = np.corrcoef(df[x1],df[y1])[0,1]
            if abs(roh) > correlation:                   #阈值确定
                x1_IV = high_IV_sorted[i][1]
                y1_IV = high_IV_sorted[j][1]
                if x1_IV > y1_IV:
                    deleted_index.append(j)
                else :
                    deleted_index.append(i)
    iv_var_ori = [high_IV_sorted[i][0] for i in range(cnt_vars) if i not in deleted_index]
    
    #以下跑 VIF，变量个数 >= 2
    if len(iv_var_ori) >= 2:
        X = np.matrix(df[iv_var_ori])
        VIF_list = [variance_inflation_factor(X, i) for i in range(X.shape[1])]
        max_VIF = max(VIF_list)
        print ('{} 的 max_VIF :{}'.format(type_, max_VIF))
    else:
        print('{}：只有一个变量'.format(type_))
    return {'iv_var_ori':iv_var_ori}